In [1]:
import os
import swiftclient.client
config = {'user':os.environ['OS_USERNAME'], 
          'key':os.environ['OS_PASSWORD'],
          'tenant_name':os.environ['OS_TENANT_NAME'],
          'authurl':os.environ['OS_AUTH_URL']}

conn = swiftclient.client.Connection(auth_version=2, **config)


In [2]:
config = {'username':os.environ['OS_USERNAME'], 
          'api_key':os.environ['OS_PASSWORD'],
          'project_id':os.environ['OS_TENANT_NAME'],
          'auth_url':os.environ['OS_AUTH_URL'],
           }
from novaclient.client import Client
nc = Client('2',**config)

In [3]:
# List containers
(response, bucket_list) = conn.get_account()
(response, bucket) = conn.get_container('tweets')
filenames = [obj['name'] for obj in bucket]



/Library/Python/2.7/site-packages/keystoneclient/service_catalog.py:196: UserWarning: Providing attr without filter_value to get_urls() is deprecated as of the 1.7.0 release and may be removed in the 2.0.0 release. Either both should be provided or neither should be provided.
  'Providing attr without filter_value to get_urls() is '


In [6]:
searchwords = ['han','hon','den','det','denna','denne','hen']


In [5]:
serverName = 'ChocolateFactory'
aKey = 'aKey'
keyPair = nc.keypairs.find(name = aKey)
flavor =  nc.flavors.find(name = 'm1.medium')
image = nc.images.find(name = 'Ubuntu Server 14.04 LTS (Trusty Tahr)')
print keyPair.name, flavor.name, image.name

aKey m1.medium Ubuntu Server 14.04 LTS (Trusty Tahr)


In [7]:
server = nc.servers.create(serverName,
                           image,
                           flavor,
                           key_name = keyPair.name
                          )
print server.name



ChocolateFactory


In [8]:
try:
    try:
        floating_ip = [ip for ip in nc.floating_ips.list() if ip.instance_id == None ].pop()
        print floating_ip
    except Exception:
        pass
    if floating_ip is None :
        floating_ip = nc.floating_ips.create('ext-net')
    server.add_floating_ip(floating_ip)
    print "ip={0}".format(floating_ip.ip)
except Exception as e:
    raise Exception("Failed to attach a floating IP to the controller.\n{0}".format(e))

<FloatingIP fixed_ip=None, id=ad671f13-cf74-4547-9700-b35af48d65cc, instance_id=None, ip=130.238.29.88, pool=ext-net>
ip=130.238.29.88


In [ ]:
import paramiko
from paramiko import SSHClient as client 
ip = floating_ip.ip
try:
    ssh = client()
    ssh.set_missing_host_key_policy(
       paramiko.AutoAddPolicy())
    ssh.connect(ip,
            username = 'ubuntu',
            key_filename ='akey.pem')
except Exception as e:
    print e
commands = [
            'sudo apt-get update -y',
            'sudo apt-get install git -y',
            'sudo apt-get install python-pip -y',
            'sudo pip install Flask',
            'sudo apt-get install rabbitmq-server -y',
            'sudo pip install celery'
           ]
for command in commands:
    try:
        print 'EXECUTEING: {}'.format(command)
        (stdin,stdout,stderr) = ssh.exec_command(command)
        print stdout.read()
        print stderr.read()
        
    except Exception as e:
        print e

In [ ]:
# Terminate all your running instances
server = nc.servers.find(name = serverName)
server.delete()
print server